# Dual-Agent Chatbot with LangGraph: Emotional vs Logical Routing

This notebook demonstrates building an intelligent chatbot system using **LangGraph** that:
- Classifies user messages as emotional or logical
- Routes messages to appropriate specialized agents
- Provides context-aware, empathetic or logical responses
- Maintains conversation state

## Architecture

```
User Input → Classifier → Router → [Therapist Agent | Logical Agent] → Response
```

The system uses:
- **Gemma 2B** model via Ollama for local inference
- **LangGraph** for workflow orchestration
- **Pydantic** for structured outputs
- **LangChain** for LLM abstractions

Let's build this step by step!

## Prerequisites

```bash
# Install dependencies
pip install langgraph langchain langchain-ollama pydantic

# Start Ollama server
ollama pull gemma:2b
ollama serve
```

---

# Imports

In [5]:
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama

## Initialize the LLM

We use **ChatOllama** to connect to a locally running Gemma 2B model. This ensures:
- ✅ Data privacy (no cloud calls)
- ✅ Fast inference
- ✅ Full control over model behavior

In [6]:
# Initialize Ollama chat model (gemma:2b) running at the local Ollama server
llm = ChatOllama(model="gemma:2b", ollama_url="http://localhost:11434")

print("Initialized LLM:", llm)

Initialized LLM: model='gemma:2b'


## Define Message Classification Schema

Using Pydantic, we enforce structured outputs from the LLM. The classifier will always return valid JSON with the message type.

In [7]:
class MessageClassifier(BaseModel):
    message_type: Literal["emotional", "logical"] = Field(
        ...,
        description="Classify if the message requires an emotional (therapist) or logical response."
    )

## Define Application State

The `State` class tracks:
- **messages**: Conversation history (maintained by LangGraph)
- **message_type**: Classification result from the classifier node

In [8]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_type: str | None

## Classifier Node

This node takes the latest user message and classifies it as either:
- **emotional**: Requires empathetic, supportive response
- **logical**: Requires factual, analytical response

The `with_structured_output()` method forces the LLM to return valid Pydantic models.

In [9]:
def classify_message(state: State):
    last_message = state["messages"][-1]
    classifier_llm = llm.with_structured_output(MessageClassifier)

    result = classifier_llm.invoke([
        {
        "role": "system", "content": "You are a message classifier that determines if a message requires an 'emotional' or 'logical' response."
        },
        {"role": "user", "content": last_message.content}
    ])

    return {"message_type": result.message_type}

## Router Node

The router makes a decision based on the classification:
- If emotional → route to therapist agent
- Otherwise → route to logical agent

This conditional routing is implemented using LangGraph's `add_conditional_edges()`.

In [10]:
def router(state: State):
    message_ttyoe = state.get("message_type", "logical")
    if message_ttyoe == "emotional":
        return {"next": "therapist"}
    
    return {"next": "logical"}

## Therapist Agent Node

A specialized agent that provides **empathetic, supportive responses**:
- Validates feelings and emotions
- Asks thoughtful, open-ended questions
- Focuses on emotional understanding
- Avoids logical problem-solving unless asked

The system prompt guides behavior while Ollama generates contextually appropriate responses.

In [19]:
def therapist_agent(state: State):
    last_message = state["messages"][-1]
    messages = [
        {"role": "system",
         "content": """You are a compassionate therapist. Focus on the emotional aspects of the user's message.
                        Show empathy, validate their feelings, and help them process their emotions.
                        Ask thoughtful questions to help them explore their feelings more deeply.
                        Avoid giving logical solutions unless explicitly asked."""
         },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}
    return {"response": response.content}

## Logical Agent Node

A specialized agent that provides **factual, evidence-based responses**:
- Focuses on information and facts
- Provides clear, direct answers
- Uses logical reasoning
- Avoids emotional interpretation

The system prompt ensures the agent stays focused on logical analysis.

In [20]:
def logical_agent(state: State):
    last_message = state["messages"][-1]
    messages = [
        {"role": "system",
         "content": """You are a purely logical assistant. Focus only on facts and information.
            Provide clear, concise answers based on logic and evidence.
            Do not address emotions or provide emotional support.
            Be direct and straightforward in your responses."""
         },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}

## Build the Graph

Create a **StateGraph** that orchestrates the workflow:

**Flow:**
1. START → classifier (analyze message type)
2. classifier → router (decide which agent)
3. router → conditional_edges (branch to therapist or logical)
4. Both agents → END (conclude turn)

LangGraph compiles this into an executable workflow.

In [21]:
graph_builder = StateGraph(State)

graph_builder.add_node("classifier", classify_message)
graph_builder.add_node("router", router)
graph_builder.add_node("therapist", therapist_agent)
graph_builder.add_node("logical", logical_agent)

graph_builder.add_edge(START, "classifier")
graph_builder.add_edge("classifier", "router")

graph_builder.add_conditional_edges(
    "router",
    lambda state: state.get("next"),
    {"therapist": "therapist", "logical": "logical"}
)

graph_builder.add_edge("therapist", END)
graph_builder.add_edge("logical", END)

graph = graph_builder.compile()

## Run the Chatbot

Execute the chatbot in an interactive loop:

**Features:**
- Maintains conversation state across turns
- Classifies each message dynamically
- Routes to appropriate agent
- Type 'exit' or 'quit' to end conversation
- Displays classification and response

Run and test with different inputs to see how it classifies and responds!

In [22]:
def run_chatbot():
    state = {"messages": [], "message_type": None}

    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting chatbot.")
            break

        state["messages"] = state.get("messages", []) + [
            {"role": "user", "content": user_input}
        ]

        state = graph.invoke(state)

        if state.get("messages") and len(state["messages"]) > 0:
            last_message = state["messages"][-1]
            print(f"Assistant: {last_message.content}")
            print(state)

if __name__ == "__main__":
    run_chatbot()

Assistant: Hello, and thank you for reaching out. It's completely understandable to feel a range of emotions when going through a difficult time. I'm here to listen and offer support in whatever way I can.

Let's take things one step at a time and explore your feelings together. What emotions are you feeling right now? Is there anything in particular that's bothering you?

Remember, it's okay to feel vulnerable and express your emotions. Processing them can be a powerful step towards healing and growth.

How would you like to describe how you're feeling right now?
{'messages': [HumanMessage(content='i am happy', additional_kwargs={}, response_metadata={}, id='5aefba68-d0e5-4700-a5d5-62bb49f38a88'), AIMessage(content="Hello, and thank you for reaching out. It's completely understandable to feel a range of emotions when going through a difficult time. I'm here to listen and offer support in whatever way I can.\n\nLet's take things one step at a time and explore your feelings together. Wh